In [4]:
import ipyleaflet as ipl
from shapely.geometry import Point
import osmnx as ox
import json
from ipywidgets import HTML
import random

# Define a bounding box for a larger area of Budapest
north = 47.5032
south = 47.4696
east = 19.0839
west = 19.0276

# Download a street network within the bounding box
G = ox.graph_from_bbox(north, south, east, west)

# Convert the graph to GeoDataFrames (nodes, edges)
node_gdf, edge_gdf = ox.graph_to_gdfs(G, nodes=True, edges=True)

# Calculate the center point of the graph
mean_lat = node_gdf.geometry.y.mean()
mean_lon = node_gdf.geometry.x.mean()
center = Point(mean_lon, mean_lat)

# Query for building polygons within the bounding box
tags = {'building': True}
buildings_gdf = ox.geometries_from_bbox(north, south, east, west, tags=tags)

# Filter to keep only Polygon and MultiPolygon geometries (building blocks)
buildings_gdf = buildings_gdf[buildings_gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]

# Save edge and building GeoDataFrames as UTF-8 encoded GeoJSON files
edge_geojson_path = "edge_network.geojson"
buildings_geojson_path = "buildings.geojson"

# Write edge GeoJSON to file with UTF-8 encoding
with open(edge_geojson_path, 'w', encoding='utf-8') as edge_file:
    edge_file.write(edge_gdf.to_json())

# Write building GeoJSON to file with UTF-8 encoding
with open(buildings_geojson_path, 'w', encoding='utf-8') as building_file:
    building_file.write(buildings_gdf.to_json())

# Now, read the GeoJSON files back in UTF-8 encoding
with open(edge_geojson_path, 'r', encoding='utf-8') as edge_file:
    edge_geojson = json.load(edge_file)

with open(buildings_geojson_path, 'r', encoding='utf-8') as building_file:
    buildings_geojson = json.load(building_file)

# Create a map
m = ipl.Map(center=(center.y, center.x), zoom=12)

# Add layer control to toggle layers
layer_control = ipl.LayersControl(position='topright')
m.add(layer_control)

# Function to create random color
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

# Add edge network layer
edge_layer = ipl.GeoJSON(
    data=edge_geojson, 
    name='Street Network', 
    hover_style={'color': 'blue', 'dashArray': '0', 'fillOpacity': 0.5},
    style_callback=random_color
)

# Define a style for the building polygons
building_style = {
    'color': 'blue',
    'weight': 2,
    'opacity': 0.8,
    'fillColor': 'blue',
    'fillOpacity': 0.5,
}

# Add building polygons layer with tooltips
buildings_layer = ipl.GeoJSON(
    data=buildings_geojson,
    name='Buildings',
    style=building_style,  # Set default style
)

# Function to create the popup content for a building
def create_building_popup(feature):
    building_type = feature['properties'].get('building', 'Unknown')
    address = feature['properties'].get('address', 'No address available')
    return f"<strong>Building Type: {building_type}</strong><br>{address}"

# Function to handle building clicks
def on_building_click(feature, **kwargs):
    coords = feature['geometry']['coordinates'][0][0]  # Get the clicked coordinates (first polygon point)
    lng, lat = coords[0], coords[1]  # Note: coords are (lon, lat)
    # Create and show a popup
    popup_content = create_building_popup(feature)
    popup_message = HTML()
    popup_message.value = popup_content
    
    # Create the popup using ipyleaflet's Popup object
    popup = ipl.Popup(
        location=(lat, lng),  # Popup location (lat, lng)
        child=popup_message,  # Pass the HTML content
        close_button=True,  # Add a close button
        auto_close=False,  # Keep it open until manually closed
        close_on_escape_key=True  # Allow closing with the escape key
    )
    
    # Add the popup to the map
    m.add(popup)

# Connect the click event to the building layer
buildings_layer.on_click(on_building_click)

# Add layers to the map
m.add(buildings_layer)
m.add(edge_layer)

# Display the map
m



/tmp/ipykernel_79025/447014009.py:15: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  G = ox.graph_from_bbox(north, south, east, west)
/tmp/ipykernel_79025/447014009.py:15: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = ox.graph_from_bbox(north, south, east, west)
/tmp/ipykernel_79025/447014009.py:27: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  buildings_gdf = ox.geometries_from_bbox(north, south, east, west, tags=ta

Map(center=[47.48406213715601, 19.052729648726345], controls=(ZoomControl(options=['position', 'zoom_in_text',…